In [1]:
import azureml
from azureml.core import Workspace, Run
import pandas as pd
import numpy
from azureml.core.model import Model
ws = Workspace.from_config()

Found the config file in: C:\Users\zhpek\Desktop\C4I\Challenge 4\aml_config\config.json


In [2]:
import logging
import os
import random
import re

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
from sklearn import datasets

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.run import Run
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

In [3]:
ws.experiments

{'train-on-remote-vm': Experiment(Name: train-on-remote-vm,
 Workspace: pekamlworkspace),
 'aks-compute-trial': Experiment(Name: aks-compute-trial,
 Workspace: pekamlworkspace),
 'sklearn-mnist': Experiment(Name: sklearn-mnist,
 Workspace: pekamlworkspace),
 'automl-local-classification': Experiment(Name: automl-local-classification,
 Workspace: pekamlworkspace),
 'tf-mnist': Experiment(Name: tf-mnist,
 Workspace: pekamlworkspace),
 'automl-pipeline-classification': Experiment(Name: automl-pipeline-classification,
 Workspace: pekamlworkspace),
 '10k-Diabetes-with-AutoML': Experiment(Name: 10k-Diabetes-with-AutoML,
 Workspace: pekamlworkspace),
 'c4ts-challenge4': Experiment(Name: c4ts-challenge4,
 Workspace: pekamlworkspace)}

In [4]:
experiment_name = 'c4ts-challenge4' # Replace this with any project name from previous cell
experiment = ws.experiments[experiment_name]
#ml_run = AutoMLRun(experiment=experiment, run_id=run_id)
selected_metric = 'AUC_weighted'
#summary_df = pd.DataFrame(index = ['Type', 'Status', 'Primary Metric', 'Iterations', 'Compute', 'Name'])
#pattern = re.compile('^AutoML_[^_]*$')
runs = list(experiment.get_runs())
#model_dict = {}
models_df = pd.DataFrame(index = [selected_metric, 'run'])
for run in runs[:2]:
    
    aml_run = AutoMLRun(experiment = experiment, run_id = run.id)
    print(run.id)
    best_run, __ = aml_run.get_output()
    metrics = best_run.get_metrics()
    
    if selected_metric in metrics:
        #model_dict[run.id]= {'run': aml_run}
        #model_dict[run.id][selected_metric] = metrics[selected_metric]
        models_df[run.id] = [metrics[selected_metric],aml_run]
        
models_df = models_df.T

AutoML_ee562a66-ffe1-4e7f-b6a9-1641937a61e8
AutoML_ece6d8e3-f37a-49c3-b601-c35261eab752


In [5]:
models_df

,AUC_weighted,run
AutoML_ee562a66-ffe1-4e7f-b6a9-1641937a61e8,0.820556,"Run(Experiment: c4ts-challenge4,\nId: AutoML_e..."
AutoML_ece6d8e3-f37a-49c3-b601-c35261eab752,0.787947,"Run(Experiment: c4ts-challenge4,\nId: AutoML_e..."


In [6]:
#model_dict

In [7]:
#acc = [model_dict[k][selected_metric] for k in model_dict]
#runlist = [model_dict[k]['run'] for k in model_dict]
#best_run = runlist[acc.index(max(acc))]
#best_run

In [8]:
run_id = models_df[selected_metric].astype(float).idxmax()
best_run = models_df.loc[run_id, 'run']
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
c4ts-challenge4,AutoML_ee562a66-ffe1-4e7f-b6a9-1641937a61e8,automl,Completed,Link to Azure Portal,Link to Documentation


In [9]:
model = best_run.register_model(description = 'AutoML Model Registration', tags = None)
model_id = best_run.model_id
model_id

Registering model AutoMLee562a66fbest


'AutoMLee562a66fbest'

In [10]:
rundl = AutoMLRun(experiment = experiment, run_id = best_run.id)
dependencies = rundl.get_run_sdk_dependencies(iteration = 2)

No issues found in the SDK package versions.


In [21]:
import pickle
from utils import scaler

artifacts = rundl.get_file_names()

for f in artifacts:
    rundl.download_file(f)

p = pickle.load(open('pca_transform.pkl','rb'))
s = pickle.load(open('scaler.pkl','rb'))
scaler_mean = list(s.mean)
scaler_var = list(s.var)
scaler_index = list(s.mean.index)

In [46]:
artifacts = ['pca_transform.pkl', 'scaler.pkl','utils.py']
artifacts

['pca_transform.pkl', 'scaler.pkl', 'utils.py']

In [47]:
model_id

'AutoMLee562a66fbest'

In [56]:
#%%writefile score_sparkml.py
#s_mean = {scaler_mean}
#s_var = {scaler_var}
#s_

#def scale(x):
#    result = (x - {scaler_mean}) / np.sqrt({scaler_var})
#    return (result)

score_script = """
 
import pickle
import json
import numpy
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model
import pandas as pd
import numpy as np
from utils import scaler

def cleanLongLat(l):
    split = l.str.split(',', expand=True)
    split = (split[0]+'.'+[i if len(i)>1 else i+'0' for i in split[1]]).astype(float)
    return(split)


def init():
    from utils import scaler

    global model
    global pca_transform
    global s
        
    s = scaler(pd.DataFrame([1,2]))
    model_id = "{model_id}"
    model_path = Model.get_model_path(model_name = model_id)
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    pca_transform = pickle.load(open('pca_transform.pkl', 'rb'))
    s = pickle.load(open('scaler.pkl','rb'))

def run(rawdata):
    try:
        df = json.loads(j_test)['data']
        df = pd.DataFrame.from_records(df)
        df['Latitude'] = cleanLongLat(df['Latitude'])
        df['Longitude'] = cleanLongLat(df['Longitude'])
        df.drop(['Machine_ID', 'District'], axis=1, inplace=True)
        df.drop('Failure_NextHour', axis = 1, inplace=True)
        df = df.apply(pd.to_numeric)
        data = pca_transform.transform(s.scale(df))
        result = best_model.predict(data) #Changed to best_model here
    except Exception as e:
        result = str(e)
        return json.dumps({{"error": result}})
    return json.dumps({{"result":result.tolist()}}) 
    
""".format(model_id = model_id)#, scaler_mean = scaler_mean, scaler_var = scaler_var)

with open("score.py", "w") as file:
    file.write(score_script)

In [57]:
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn','pandas'], pip_packages=['azureml-sdk[automl]'])

conda_env_file_name = 'myenv.yml'
myenv.save_to_file('.', conda_env_file_name)

'myenv.yml'

In [58]:
%%time
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(runtime= "python",
                                                  execution_script = 'score.py',
                                                  conda_file = conda_env_file_name,
                                                  dependencies = artifacts,
                                                  description = "Image for c4ts Challenge 4")

image = Image.create(name = "c4tsimg2",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

image.wait_for_creation(show_output = True)

if image.creation_state == 'Failed':
    print("Image build log at: " + image.image_build_log_uri)

Creating image
Running................................................
SucceededImage creation operation finished for image c4tsimg2:19, operation "Succeeded"
Wall time: 5min 21s


In [59]:
image.image_location

'pekamlwoacrjcvuohfw.azurecr.io/c4tsimg2:19'

In [60]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "c4ts4",  "method" : "automl"}, 
                                               description='C4TS Challenge 4')

In [61]:
%%time
from azureml.core.webservice import Webservice

service = Webservice.deploy_from_image(workspace=ws,
                                       name='c4ts-svc',
                                       deployment_config=aciconfig,
                                       image=image)

service.wait_for_deployment(show_output=True)
print(service.state)

Creating service
Running...............................
FailedACI service creation operation finished, operation "Failed"
Service creation polling reached terminal state, unexpected error response:
{'code': 'AciDeploymentFailed', 'message': 'Aci Deployment failed', 'details': [{'code': 'CrashLoopBackOff', 'message': "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance c4ts-svc.\nYou can also try to run image pekamlwoacrjcvuohfw.azurecr.io/c4tsimg2:19 locally. Please refer to http://aka.ms/debugimage for more information."}]}
Unhealthy
Wall time: 3min 29s


In [62]:
print(service.get_logs())

2019-01-09T08:53:56,126111042+00:00 - iot-server/run 
2019-01-09T08:53:56,126777662+00:00 - rsyslog/run 
2019-01-09T08:53:56,128219804+00:00 - gunicorn/run 
ok: run: gunicorn: (pid 14) 0s
ok: run: nginx: (pid 12) 0s
ok: run: rsyslog: (pid 13) 0s
2019-01-09T08:53:56,133294053+00:00 - nginx/run 
ok: run: rsyslog: (pid 13) 0s
ok: run: rsyslog: (pid 13) 0s
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2019-01-09T08:53:56,309872922+00:00 - iot-server/finish 1 0
2019-01-09T08:53:56,311016455+00:00 - Exit code 1 is normal. Not restarting iot-server.
{"timestamp": "2019-01-09T08:53:56.560825Z", "message": "Starting gunicorn 19.6.0", "host": "wk-caas-895de745f7ba4271867822c90175767b-28255d2ca56dd042d2a6a2", "path": "/opt/miniconda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "msg": "Starting gunicorn %s", "stack_info": null}
{"timestamp": "2019-01-09T08:

In [ ]:
df = pd.read_csv('AssetData_Historical.csv')

In [ ]:
import csv
import json
with open('AssetData_Historical.csv') as f:
    reader = csv.DictReader(f)
    titles = reader.fieldnames
    j_test = json.dumps({'data':[row for row in reader]})
    
result = service.run(input_data = j_test)

In [ ]:
import csv
import json
with open('AssetData_Historical.csv') as f:
    reader = csv.DictReader(f)
    titles = reader.fieldnames
    j_test = json.dumps({'data':[row for row in reader]})
    
def run(rawdata):
    try:
        df = json.loads(j_test)['data']
        df = pd.DataFrame.from_records(df)
        df['Latitude'] = cleanLongLat(df['Latitude'])
        df['Longitude'] = cleanLongLat(df['Longitude'])
        df.drop(['Machine_ID', 'District'], axis=1, inplace=True)
        df.drop('Failure_NextHour', axis = 1, inplace=True)
        df = df.apply(pd.to_numeric)
        data = pca_transform.transform(scaler.scale(df))
        result = best_model.predict(data) #Changed to best_model here
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    return json.dumps({"result":result.tolist()}) #removed double brackets here

run(j_test)

In [ ]:
df = json.loads(j_test)['data']
df = pd.DataFrame.from_records(df)
df['Latitude'] = cleanLongLat(df['Latitude'])
df['Longitude'] = cleanLongLat(df['Longitude'])
df.drop(['Machine_ID', 'District'], axis=1, inplace=True)
df.drop('Failure_NextHour', axis = 1, inplace=True)
data = pca_transform.transform(scaler.scale(df))
result = best_model.predict(data) #Changed to best_model here

df = json.loads(j_test)['data']
df = pd.DataFrame.from_records(df)
df['Latitude'] = cleanLongLat(df['Latitude'])
df['Longitude'] = cleanLongLat(df['Longitude'])
df.drop(['Machine_ID', 'District'], axis=1, inplace=True)
df.drop('Failure_NextHour', axis = 1, inplace=True)
df = df.apply(pd.to_numeric)
data = pca_transform.transform(scaler.scale(df))

df.apply(pd.to_numeric)
#data = pca_transform.transform(scaler.scale(df))

#pd.DataFrame.from_records(data['messages']).map.apply(pd.Series)
pd.DataFrame.from_records(j_load)

from utils import scaler, cleanLongLat

pca_transform = pickle.load(open('pca_transform.pkl','rb'))
scaler = pickle.load(open('scaler.pkl','rb'))


#s = scaler_obj.scale(X)
#p = pca_transform.transform(s)
#j = json.dumps({'data':p.tolist()})

#df = json.loads(j_test)['data']
#df = pd.DataFrame.from_records(df)
#df['Latitude'] = cleanLongLat(df['Latitude'])
#df['Longitude'] = cleanLongLat(df['Longitude'])
#df.drop(['Machine_ID', 'District'], axis=1, inplace=True)
#df.drop('Failure_NextHour', axis = 1, inplace=True)
#df = df.apply(pd.to_numeric)
#data = pca_transform.transform(scaler.scale(df))